In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import altair as alt
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

# show all columns in the dataframe
pd.set_option('max_columns', None)

# Lees predictie functies

In [73]:
# Zorg er voor dat de veranderingen in verbruik_predictie ook meegenomen worden
from imp import reload
import verbruik_predictie as vp
reload(vp)


<module 'verbruik_predictie' from 'c:\\Users\\ericr\\OneDrive - Actondata\\Projecten\\jads_enexis\\notebooks\\verbruik_predictie.py'>

# Lees kleinverbruik data

In [7]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [8]:
# kleinverbruikgegevens gegevens inlezen
df = pd.read_hdf('kleinverbruikgegevens_data.h5')

#Delete 2021 data by keeping JAAR < 2021
df = df[df['JAAR'] < 2021]

# Voeg Features toe
Voeg profiel toe op basis van SOORT_AANSLUITING en de SJV_TOTAAL per profiel

In [9]:
# Spoor 2. Voeg profiel toe aan verbruiksgegevens. SOORT_AANSLUITING bepaald het profiel
# Omdat de E2 profielen in de NEDU set niet consistent zijn (verschil tussen 2010-2017 en 2018-), gebruiken we deze niet
profiel_E1 = ['1X25','3X25', '1X20', '1x25', '3x25', '1x20'] # de rest is profiel E2

# Voeg in de kleinverbruikgegevens, het bijpassende profiel voor de soort aansluiting toe
def vervang_door_profiel(aansluiting, postcode, percentage):
    # Bepaal basisprofiel
    profiel = 'E1' # if aansluiting in profiel_E1 else 'E2' # Gebruiken als we wel een goed E2 profiel hebben

    # Een laag tarief percentage lager dan 50% zetten we in een A profiel
    lh_profiel = 'A' if percentage < 50 else 'B'

    # Bepaal welk laag tarief gebied de postcode zit
    if profiel == 'E1' and lh_profiel == 'B':
        postcode_area = int(postcode[:2]) # PC4
        # Postcodes < 65 is Noord-Brabant of Limburg
        if postcode_area < 65:
            lh_profiel = 'C'
    return profiel + lh_profiel

df["PROFIEL"] = np.vectorize(vervang_door_profiel)(df.SOORT_AANSLUITING, df.POSTCODE_VAN, df.SJV_LAAG_TARIEF_PERC)
df.PC4 = df.PC4.astype('int')
print (f'#E1A = {df[df.PROFIEL == "E1A"].PROFIEL.count()}')
print (f'#E1B = {df[df.PROFIEL == "E1B"].PROFIEL.count()}')
print (f'#E1C = {df[df.PROFIEL == "E1C"].PROFIEL.count()}')

#E1A = 956271
#E1B = 52567
#E1C = 257304


In [10]:
# Voeg features toe per PC6
df["E1A_TOTAAL"] = df[df.PROFIEL == 'E1A'].SJV_TOTAAL
df["E1B_TOTAAL"] = df[df.PROFIEL == 'E1B'].SJV_TOTAAL
df["E1C_TOTAAL"] = df[df.PROFIEL == 'E1C'].SJV_TOTAAL
df["WEIGHTED_LEVERINGSRICHTING_PERC"] = df.AANSLUITINGEN_AANTAL * df.LEVERINGSRICHTING_PERC

# Rol op tot PC4

In [11]:
# Rol op tot PC4
df_verbruik = df.groupby(['PC4','JAAR']).agg({'SJV_TOTAAL':'sum', 'E1A_TOTAAL' : 'sum', 'E1B_TOTAAL' : 'sum', 'E1C_TOTAAL': 'sum', 'AANSLUITINGEN_AANTAL':'sum', 'WEIGHTED_LEVERINGSRICHTING_PERC': 'sum'})
df_verbruik['LEVERINGSRICHTING_PERC'] = df_verbruik['WEIGHTED_LEVERINGSRICHTING_PERC'] / df_verbruik['AANSLUITINGEN_AANTAL']
df_verbruik = df_verbruik.drop(columns=['WEIGHTED_LEVERINGSRICHTING_PERC'])
df_verbruik

SJV_TOTAAL  E1A_TOTAAL  E1B_TOTAAL  E1C_TOTAAL  \
PC4  JAAR                                                   
4251 2010     21802.0     17114.0         0.0      4688.0   
     2011     23084.0     18179.0         0.0      4905.0   
     2012     24006.0     18172.0         0.0      5834.0   
     2013     23728.0     18421.0         0.0      5307.0   
     2014     23623.0     22610.0         0.0      1013.0   
...               ...         ...         ...         ...   
9999 2016        56.0        56.0         0.0         0.0   
     2017        53.0        53.0         0.0         0.0   
     2018       174.0       174.0         0.0         0.0   
     2019       163.0       163.0         0.0         0.0   
     2020       155.0       155.0         0.0         0.0   

           AANSLUITINGEN_AANTAL  LEVERINGSRICHTING_PERC  
PC4  JAAR                                                
4251 2010                  4325               99.949133  
     2011                  4413               99.950147  
     2012                  4486               99.848417  
     2013                  4512               99.578901  
     2014                  4544               98.932218  
...                         ...                     ...  
9999 2016                    21               81.000000  
     2017                    21               71.000000  
     2018                    37               64.865676  
     2019                    37               62.164054  
     2020                    37               54.052432  

[17130 rows x 6 columns]

In [12]:
# Verhuis de multi-level index naar kolommen en maak de index een simpele range van getallen
df_verbruik['PC4'] = df_verbruik.index.get_level_values('PC4')
df_verbruik['JAAR'] = df_verbruik.index.get_level_values('JAAR')
df_verbruik.index = range(len(df_verbruik))
df_verbruik.SJV_TOTAAL = df_verbruik.SJV_TOTAAL.astype('int')
df_verbruik.E1A_TOTAAL = df_verbruik.E1A_TOTAAL.astype('int')
df_verbruik.E1B_TOTAAL = df_verbruik.E1B_TOTAAL.astype('int')
df_verbruik.E1C_TOTAAL = df_verbruik.E1C_TOTAAL.astype('int')
df_verbruik

,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4,JAAR
0,21802,17114,0,4688,4325,99.949133,4251,2010
1,23084,18179,0,4905,4413,99.950147,4251,2011
2,24006,18172,0,5834,4486,99.848417,4251,2012
3,23728,18421,0,5307,4512,99.578901,4251,2013
4,23623,22610,0,1013,4544,98.932218,4251,2014
...,...,...,...,...,...,...,...,...
17125,56,56,0,0,21,81.000000,9999,2016
17126,53,53,0,0,21,71.000000,9999,2017
17127,174,174,0,0,37,64.865676,9999,2018
17128,163,163,0,0,37,62.164054,9999,2019


# Voeg predictie toe voor de jaren 2021 tot 2023

In [67]:
# Voorspel het verbruik met lineaire regressie
df_pred = vp.predict_verbruik_lr(df_verbruik, predict_type='mid')
df_pred[df_pred.PC4 == 4251]

100%|██████████| 1645/1645 [00:13<00:00, 119.15it/s]


,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4,JAAR
0,22442.327273,20900.509091,0.0,1541.218182,4797.254545,92.771728,4251,2021
1,22369.336364,21107.381818,0.0,1261.254545,4835.509091,92.011754,4251,2022
2,22296.345455,21314.254545,0.0,981.290909,4873.763636,91.251780,4251,2023


In [74]:
accuracy = vp.calc_accuracy_lr(df_verbruik)
accuracy

{'r2_train': 8.410242799850032e-06,
 'r2_test': -0.003664233452043586,
 'MSE_train': 54704272.756639056,
 'MSE test': 50657203.6206131,
 'RMSE train': 7396.233687265368,
 'RMSE test': 7117.387415380246,
 'MAE train': 5858.4002013110785,
 'MAE test': 5728.186721532426,
 'MAPE train': 503.7932697301765,
 'MAPE test': 599.2498281955808}

In [254]:
# Check de merge voor postcode 4251
df_verbruik_en_pred = pd.merge(df_verbruik, df_pred, how='outer')
df_verbruik_en_pred[df_verbruik_en_pred.PC4 == 4251].head(14)

,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4,JAAR
0,21802.000000,17114.000000,0.0,4688.000000,4325.000000,99.949133,4251,2010
1,23084.000000,18179.000000,0.0,4905.000000,4413.000000,99.950147,4251,2011
2,24006.000000,18172.000000,0.0,5834.000000,4486.000000,99.848417,4251,2012
3,23728.000000,18421.000000,0.0,5307.000000,4512.000000,99.578901,4251,2013
4,23623.000000,22610.000000,0.0,1013.000000,4544.000000,98.932218,4251,2014
5,22848.000000,21964.000000,0.0,884.000000,4570.000000,98.370897,4251,2015
6,22645.000000,21200.000000,0.0,1445.000000,4581.000000,97.923161,4251,2016
7,22742.000000,19830.000000,0.0,2912.000000,4661.000000,97.329758,4251,2017
8,22680.000000,20056.000000,0.0,2624.000000,4713.000000,95.501944,4251,2018
9,22391.000000,19537.000000,0.0,2854.000000,4720.000000,92.817822,4251,2019


# Integratie met NEDU profielen - Deze code vervangen door code spoor 1 (Marcel)

In [255]:
# Combineer met NEDU profielen. 
df_nedu_profielen = pd.read_hdf('nedu_files.h5')
df_nedu_profielen_origineel = df_nedu_profielen

In [256]:
df_nedu_profielen['jaar'] = df_nedu_profielen.DatumTijd.dt.year
df_nedu_profielen['maand'] = df_nedu_profielen.DatumTijd.dt.month
df_nedu_profielen['dag'] = df_nedu_profielen.DatumTijd.dt.day

df_nedu_profielen = df_nedu_profielen.groupby(['jaar','maand','dag']).agg({'E1A':'sum', 'E1B':'sum', 'E1C':'sum', 'E2A':'sum', 'E2B':'sum'})

def maak_datum(jaar,maand,dag):
    return format(jaar,'04d') + '-' + format(maand,'02d') + '-' + format(dag,'02d'), jaar
df_nedu_profielen.index, df_nedu_profielen["JAAR"] = np.vectorize(maak_datum)(df_nedu_profielen.index.get_level_values('jaar'), df_nedu_profielen.index.get_level_values('maand'), df_nedu_profielen.index.get_level_values('dag'))
df_nedu_profielen.index = pd.to_datetime(df_nedu_profielen.index)

# Herorganiseren kolommen. Index is een rijteller. Datum moet een datetime worden
df_nedu_profielen = df_nedu_profielen[df_nedu_profielen.JAAR < 2021]
df_nedu_profielen = df_nedu_profielen.drop(columns=['E2A', 'E2B'])
df_nedu_profielen['DATUM'] = df_nedu_profielen.index
df_nedu_profielen.index = range(len(df_nedu_profielen))
df_nedu_profielen.DATUM = pd.to_datetime(df_nedu_profielen.DATUM, format='%d-%m-%Y %H:%M')

# Einde tijdelijke code - Vervangen door code spoor 1 (Marcel)

In [ ]:
# Combinatiespoor
Combineer de dataframes van de NEDU profielen met voorspelling en het dataframe van de kleinverbruikgegevens inclusief voorspelling

In [257]:
# Combineer NEDU profielen met de kleinverbruikgegevens
df_combined = pd.merge(df_nedu_profielen, df_verbruik, on=['JAAR'], how='left')

In [259]:
df_combined[df_combined.PC4 == 4251].head(14)

,E1A,E1B,E1C,JAAR,DATUM,SJV_TOTAAL,E1A_TOTAAL,E1B_TOTAAL,E1C_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC,PC4
0,0.003231,0.003311,0.003430,2010,2010-01-01,21802,17114,0,4688,4325,99.949133,4251
1550,0.003448,0.003514,0.003620,2010,2010-01-02,21802,17114,0,4688,4325,99.949133,4251
3100,0.003314,0.003448,0.003394,2010,2010-01-03,21802,17114,0,4688,4325,99.949133,4251
4650,0.003275,0.003303,0.003323,2010,2010-01-04,21802,17114,0,4688,4325,99.949133,4251
6200,0.003232,0.003175,0.003221,2010,2010-01-05,21802,17114,0,4688,4325,99.949133,4251
7750,0.003287,0.003260,0.003194,2010,2010-01-06,21802,17114,0,4688,4325,99.949133,4251
9300,0.003249,0.003249,0.003284,2010,2010-01-07,21802,17114,0,4688,4325,99.949133,4251
10850,0.003326,0.003277,0.003315,2010,2010-01-08,21802,17114,0,4688,4325,99.949133,4251
12400,0.003397,0.003532,0.003541,2010,2010-01-09,21802,17114,0,4688,4325,99.949133,4251
13950,0.003383,0.003393,0.003414,2010,2010-01-10,21802,17114,0,4688,4325,99.949133,4251


In [260]:
# Voeg nieuwe kolom VERBRUIK toe die de som is van het verbruik van de verschillende profielen
df_combined['VERBRUIK'] = df_combined.E1A * df_combined.E1A_TOTAAL + df_combined.E1B * df_combined.E1B_TOTAAL + df_combined.E1C * df_combined.E1C_TOTAAL

# Gooi de tussenkolommen weg. Die hebben we niet meer nodig
df_combined = df_combined.drop(columns=['E1A','E1B','E1C','E1A_TOTAAL','E1B_TOTAAL','E1C_TOTAAL','JAAR'])

In [261]:
# cross check. Het totaal van een profiel over een heel jaar moet 1 zijn. Dat betekent dat het totaal van de verbruiken gelijk moet zijn aan SJV_TOTAAL
print (f"Totaal verbruik = {df_combined[(df_combined.DATUM < datetime(2011,1,1)) & (df_combined.PC4 == 4251)].agg({'VERBRUIK':'sum'}).values[0]}")
print (f"SJV totaal = {df_combined[(df_combined.DATUM < datetime(2011,1,1)) & (df_combined.PC4 == 4251)].SJV_TOTAAL.values[0]}")

Totaal verbruik = 21800.70682874
SJV totaal = 21802
